In [1]:
import load_data as ld

Loader = ld.DataLoader()

train_df, val_df, test_df = Loader.load_all_three_dfs(train_size=5000, val_size=2000, test_size=2000)

In [2]:
import pretrained_embedder as pe
embedder = pe.PretrainedEmbedder("esm1_t6_43M_UR50S", sequence_length=512)

c:\Users\Ryan\miniconda3\envs\bio-transformers-dev\lib\site-packages\ray\autoscaler\_private\cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [9]:
for i in range(int(5000/ 100)):
    start = i * 100
    end = (i+1) * 100

    sample = train_df.iloc[start:end, :]

    esm_train = embedder.embed_sequences(input_df=sample) 

    file_name = 'embeddings/train' + str(i) + '.csv'
    esm_train.to_csv(file_name)

100%|██████████| 100/100 [01:51<00:00,  1.12s/it]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:50<00:00,  1.10s/it]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:51<00:00,  1.12s/it]


...

(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:52<00:00,  1.13s/it]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:50<00:00,  1.11s/it]

(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


In [10]:
for i in range(int(2000/ 100)):
    start = i * 100
    end = (i+1) * 100

    sample = val_df.iloc[start:end, :]

    esm_val = embedder.embed_sequences(input_df=sample) 

    file_name = 'embeddings/val' + str(i) + '.csv'
    esm_val.to_csv(file_name)

    sample = test_df.iloc[start:end, :]
    esm_test = embedder.embed_sequences(input_df=sample) 

    file_name = 'embeddings/test' + str(i) + '.csv'
    esm_test.to_csv(file_name)

    

100%|██████████| 100/100 [01:44<00:00,  1.04s/it]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:13<00:00,  1.36it/s]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:44<00:00,  1.04s/it]


...

(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 100/100 [01:43<00:00,  1.04s/it]

(100, 768)
<class 'pandas.core.frame.DataFrame'>
(100, 5)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
esm_val= embedder.embed_sequences(input_df=val_df)
esm_test = embedder.embed_sequences(input_df=test_df)

esm_val.to_csv('esm_val.csv')
esm_test.to_csv('esm_test.csv')